In [1]:
import os

In [2]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
len(list_file)

1401

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import json

In [6]:
for file in tqdm(list_file, desc='Progress', ncols=77):
    with open('./part/' + file, 'r', encoding='UTF-8') as f:
        html = f.read()
        
        # = = = = = = = = = = = = = = =
        
        soup = BeautifulSoup(html, 'lxml')
        
        # = = = = = = = = = = = = = = =
        
        html = etree.HTML(str(soup))

        brand = html.xpath('//a[@itemprop="url"]/span[@itemprop="name"]/text()')[0].strip()

        # = = = = = = = = = = = = = = =
        
        title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        part_number = html.xpath('//dd[@itemprop="sku"]/text()')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        part_type = html.xpath('//dd[@data-product-parttype]/text()')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        url = html.xpath('//meta[@property="og:url"]/@content')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        src = html.xpath('//div[@class="productView-img-container"]/a/@href')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        df_temp = pd.DataFrame([{'序号': int(file.replace('.txt', '')),
                                 'Part_Type': part_type,
                                 'Part_Number': part_number,
                                 'Brand': brand,
                                 'Title': title,
                                 'Application_Summary': '',
                                 'Vehicle_1': '',
                                 'Vehicle_2': '',
                                 'OE': '',
                                 'Pic': '',
                                 'Url': url,
                                 'Src': ''}])
        
        # = = = = = = = = = = = = = = =
        
        list_name_value = json.loads(html.xpath('//div[@id="addon"]/@data-custom-fields')[0].strip())

        for name_value in list_name_value:
            if name_value['name'].strip() == 'Application Summary':
                df_temp.loc[0, 'Application_Summary'] = '\n'.join([vehicle.strip() for vehicle in name_value['value'].split(';')])
            else:
                df_temp.loc[0, name_value['name'].strip()] = name_value['value'].strip()
        
        # = = = = = = = = = = = = = = =
        
        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =
        
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df

Progress: 100%|██████████████████████████| 1401/1401 [01:35<00:00, 14.62it/s]


KeyError: 'Page'